# Configure PWM

This example shows you how to start a pulse width modulation on a pin and change it in real time over a serial connection. The steps are nearly identical over a WiFi connection. See start_webserver_over_serial.ipynb for details of starting a WiFi connection.

This example works best if you view pin 15 on an oscilloscope while running.

## Import packages

In [12]:
import sys
import os
import time

# Get the current working directory (the directory where the notebook is located)
notebook_dir = os.getcwd()

# Add the 'source' folder to the path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', 'source')))

# Now you can import from 'source'
from serial_controller import SerialController
from serial_command_sender import SerialCommandSender
from web_controller import WebController
from web_command_sender import WebCommandSender


## Initialize the Serial Command Sender and Control PWM

Note: make sure nothing else is holding the serial line (for instance, a micropico VSC project). You may need to disconnect and reconnect the Pico's USB serial link.

In [ ]:
# Initialize the serial controller with the appropriate port. This must be done in every cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.
SERIAL_PORT = "COM6" #Find the serial port your RP Pico W is connected to, can use the MicroPico: Help > List Serial Ports in VSC's MicroPico package. Hit F1 to search for command in VSC
serial_controller = SerialController(port=SERIAL_PORT, debug=False)  # Adjust port as needed
serial_controller.connect()

# Initialize the command sender
serial_command_sender = SerialCommandSender(serial_controller)

# List available commands
print("Available commands:")
response = serial_command_sender.send_command('list_commands')
response_with_tabs = "\n".join("\t" + line for line in response.splitlines())
print(response_with_tabs) # Print the response with tabs to make it easier to read

# Create a PWM hardware, in this case we'll use pin 15
settings = {
    'pin_number': 15, # Use pin 15
    'frequency': 1000, # Set the frequency to 1 kHz
    'duty_cycle': 0.5, # Set the duty cycle to 50%
    #'pulse_width_ns': 10000 # Set the pulse width to 10 microseconds (alternative to duty_cycle)
}
print(serial_command_sender.send_command('create', 'pwm', settings, "test_pwm"))

# Start the PWM hardware
print(serial_command_sender.send_command('start', 'test_pwm'))

time.sleep(5)  # Wait for 5 seconds

# Change PWM parameters on the fly
new_settings = {
    'frequency': 2000,
    'pulse_width_ns': 100000 # Set the pulse width to 100 microseconds (alternative to duty_cycle)
}
print(serial_command_sender.send_command('apply_hardware_settings', 'test_pwm', new_settings))

time.sleep(5)  # Wait for 5 seconds

# Stop the PWM hardware
print(serial_command_sender.send_command('stop', 'test_pwm'))

# Delete the PWM hardware from the list of active hardware
# If you don't delete the hardware, you won't be able to create a new hardware with the same name
# The hardware configuration lives for the lifetime of the "firmware", so in principle
# you could not delete the hardware and start it in a completely different python instance
print(serial_command_sender.send_command('delete', 'test_pwm'))


serial_controller.close() #Serial controller must be closed in each cell in jupyter notebooks to avoid holding the serial line. There is no such restriction in a python script.


Connected to COM6
Available commands:
Command sent: run_command("list_commands")
	apply_hardware_settings(hardware_id, settings)
	get_all_config()
	set_config(config_key, value)
	connect_wifi()
	start_webserver()
	stop_webserver()
	set_wifi_credentials(ssid, password)
	start(hardware_id)
	get_config(config_key)
	delete_config_key(config_key)
	list_commands()
	apply_webserver_settings(settings)
	delete(hardware_id)
	stop(hardware_id)
	create(settings)
	save_config()
	load_config()
	apply_config()
Command sent: run_command("create","pwm", {"pin_number": 15, "frequency": 1000, "duty_cycle": 0.5}, "test_pwm")
Created PWM hardware with ID test_pwm.
Command sent: run_command("start","test_pwm")
Hardware with ID test_pwm started.
Command sent: run_command("apply_hardware_settings","test_pwm", {"frequency": 2000, "pulse_width_ns": 100000})
Settings applied to hardware ID test_pwm: {'pulse_width_ns': 100000, 'frequency': 2000}
Command sent: run_command("stop","test_pwm")
Component on hardware I